In [76]:
import sys
!{sys.executable} -m pip install pyarrow
import fastf1
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Setup
cache_path = Path('cache')
cache_path.mkdir(exist_ok=True)
fastf1.Cache.enable_cache(str(cache_path))

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [77]:
def collect_and_save_season(year, output_format="csv"):
    output_dir = Path(f"data/raw/{year}")
    output_dir.mkdir(parents=True, exist_ok=True)

    all_races = []
    schedule = fastf1.get_event_schedule(year)
    num_races = len(schedule)
    print(f"Collecting {year} season ({num_races} races)...")

    for round_num in range(1, num_races + 1):
        try:
            print(f"  Round {round_num}...", end="")
            session = fastf1.get_session(year, round_num, "R")
            session.load(laps=True, telemetry=False, weather=True, messages=True)

            keep = ["Abbreviation","FullName","TeamName","GridPosition","Position","Points","Status","Time","Laps"]
            race = session.results[keep].copy()
            race["Year"] = year
            race["Round"] = round_num
            race["TrackName"] = session.event["EventName"]
            race["Country"] = session.event["Country"]

            if session.laps is not None and not session.laps.empty:
                lap_agg = (
                    session.laps.groupby("Driver")
                    .agg(
                        AvgLapTime=("LapTime", lambda s: s.dt.total_seconds().mean()),
                        NumPitStops=("PitInTime", lambda s: s.notna().sum()),
                    )
                    .reset_index()
                    .rename(columns={"Driver": "Abbreviation"})
                )
                race = race.merge(lap_agg, on="Abbreviation", how="left")

            if session.weather_data is not None and not session.weather_data.empty:
                race["AvgTrackTemp"] = float(session.weather_data["TrackTemp"].mean())
                race["AvgAirTemp"] = float(session.weather_data["AirTemp"].mean())
                race["RainDuringRace"] = bool(session.weather_data["Rainfall"].any())

            sc = False
            rcm = getattr(session, "race_control_messages", None)
            if rcm is not None and not rcm.empty:
                sc = bool(rcm["Message"].str.contains("SAFETY CAR", na=False).any())
            race["SafetyCarDeployed"] = sc

            all_races.append(race)
            print(f" ✓ {session.event['EventName']}")
        except Exception as e:
            print(f" ✗ Error: {e}")
            continue

    if not all_races:
        return pd.DataFrame()

    combined = pd.concat(all_races, ignore_index=True)
    output_file = output_dir / (f"{year}_season_extended.csv" if output_format == "csv" else f"{year}_season_extended.parquet")
    if output_format == "csv":
        combined.to_csv(output_file, index=False)
    else:
        combined.to_parquet(output_file, index=False)
    print(f"Saved {len(all_races)} races to {output_file}")
    return combined


In [78]:
# Collect 2021 and 2022 data
for year in [2021, 2022]:
    data = collect_and_save_season(year, output_format='csv')
    if not data.empty:
        print(f"{year}: Collected {len(data)} driver entries from {data['Round'].nunique()} races\n")

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.6.0]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...


  Round 1...

req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

 ✓ Bahrain Grand Prix
  Round 2...

req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for _extended_timing_data. Loading data...
_api           INFO 	Fetching timing data...
_api           INFO 	Parsing timing data...
req            INFO 	Data has been written to cache!

...

 ✗ Error: The data you are trying to access has not been loaded yet. See `Session.load`
  Round 21...

logger      WARNING 	Failed to load session info data!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
core        WARNING 	Failed to load extended driver information!
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
core        WARNING 	Failed to load driver list and session results!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
logger      WARNING 	Failed to load session status data!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
logger      WARNING 	Failed to load total lap count!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
logger      WARNIN

 ✗ Error: The data you are trying to access has not been loaded yet. See `Session.load`
  Round 22...

logger      WARNING 	Failed to load session info data!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
core        WARNING 	Failed to load extended driver information!
logger      WARNING 	Failed to load result data from Ergast!
core        WARNING 	No result data for this session available on Ergast! (This is expected for recent sessions)
core        WARNING 	Failed to load driver list and session results!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
logger      WARNING 	Failed to load session status data!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
logger      WARNING 	Failed to load total lap count!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
logger      WARNIN

 ✗ Error: The data you are trying to access has not been loaded yet. See `Session.load`
  Round 23... ✗ Error: Invalid round: 23
  Round 24... ✗ Error: Invalid round: 24
Saved 12 races to data/raw/2022/2022_season_extended.csv
2022: Collected 240 driver entries from 12 races



In [79]:
def load_season_data(year):
    p = Path(f"data/raw/{year}")
    for fn in [f"{year}_season_extended.csv", f"{year}_season_extended.parquet",
               f"{year}_season.csv", f"{year}_season.parquet"]:
        f = p / fn
        if f.exists():
            return pd.read_csv(f) if f.suffix==".csv" else pd.read_parquet(f)
    print(f"No data files found for {year}")
    return pd.DataFrame()


def load_multiple_seasons(years):
    all_seasons = []
    for year in years:
        season_data = load_season_data(year)
        if not season_data.empty:
            all_seasons.append(season_data)
    
    if all_seasons:
        return pd.concat(all_seasons, ignore_index=True)
    return pd.DataFrame()

In [80]:
from pathlib import Path
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 0)

def load_season_data(year: int) -> pd.DataFrame:
    base = Path(f"data/raw/{year}")
    for f in [base / f"{year}_season_extended.parquet",
              base / f"{year}_season_extended.csv",
              base / f"{year}_season.parquet",
              base / f"{year}_season.csv"]:
        if f.exists():
            return pd.read_parquet(f) if f.suffix == ".parquet" else pd.read_csv(f)
    print(f"[warn] No data files found for {year}")
    return pd.DataFrame()

def load_multiple_seasons(years) -> pd.DataFrame:
    frames = [load_season_data(y) for y in years]
    frames = [f for f in frames if not f.empty]
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

train_data = load_multiple_seasons([2021, 2022])
print(f"\nCombined training data: {train_data.shape}")

for c in ["GridPosition","Position","Points","Laps","Year","Round","AvgLapTime","NumPitStops","AvgTrackTemp","AvgAirTemp"]:
    if c in train_data.columns:
        train_data[c] = pd.to_numeric(train_data[c], errors="coerce")

train_data["Time"] = pd.to_timedelta(train_data.get("Time", pd.Series([np.nan]*len(train_data))), errors="coerce")

for bc in ["RainDuringRace","SafetyCarDeployed"]:
    if bc in train_data.columns:
        train_data[bc] = train_data[bc].astype("boolean")

is_winner = train_data["Position"] == 1.0
train_data["GapToWinner_s"] = np.where(is_winner, 0.0, train_data["Time"].dt.total_seconds())

train_data["WinnerRaceTime_s"] = (
    train_data.groupby(["Year","Round"])["Time"]
              .transform(lambda s: s.max().total_seconds() if s.notna().any() else np.nan)
)

train_data["Time_s"] = train_data["WinnerRaceTime_s"] + train_data["GapToWinner_s"]

def _fmt_hms_ms(sec):
    if pd.isna(sec): return np.nan
    sec = float(sec)
    ms = int(round((sec - int(sec)) * 1000))
    if ms == 1000:
        ms = 0
        sec = int(sec) + 1
    s = int(sec) % 60
    m = (int(sec) // 60) % 60
    h = int(sec) // 3600
    return f"{h:02d}:{m:02d}:{s:02d}.{ms:03d}"

train_data["Time_str"] = train_data["Time_s"].apply(_fmt_hms_ms)

train_data = train_data.rename(columns={"Position":"RacePosition","GridPosition":"QualifyingPosition"})

for cat in ["Abbreviation","FullName","TeamName","Status","TrackName","Country"]:
    if cat in train_data.columns:
        train_data[cat] = train_data[cat].astype("string").fillna(pd.NA).str.strip()

if "Status" in train_data.columns:
    train_data["FinishedFlag"] = train_data["Status"].str.lower().eq("finished").astype("Int8")

train_data = train_data.drop(columns=["Time"])

cols_order = [
    "Year","Round","TrackName","Country",
    "Abbreviation","FullName","TeamName","Status",
    "QualifyingPosition","RacePosition","Points","Laps",
    "AvgLapTime","NumPitStops","AvgTrackTemp","AvgAirTemp",
    "RainDuringRace","SafetyCarDeployed",
    "Time_s","Time_str","GapToWinner_s","WinnerRaceTime_s",
    "FinishedFlag"
]
train_data = train_data[[c for c in cols_order if c in train_data.columns]]

out_dir = Path("data/processed")
out_dir.mkdir(parents=True, exist_ok=True)
csv_path = out_dir / "f1_2021_2022_processed.csv"
parq_path = out_dir / "f1_2021_2022_processed.parquet"

train_data.to_csv(csv_path, index=False)
try:
    train_data.to_parquet(parq_path, index=False)
except Exception as e:
    print(f"[warn] parquet save failed: {e}")

print(f"\nSaved processed data to:\n- {csv_path}\n- {parq_path if parq_path.exists() else '(parquet not written)'}")
print("\nPreview:")
print(train_data.head(8).to_string(index=False))



Combined training data: (680, 19)
[warn] parquet save failed: A type extension with name pandas.period already defined

Saved processed data to:
- data/processed/f1_2021_2022_processed.csv
- (parquet not written)

Preview:
 Year  Round          TrackName Country Abbreviation         FullName        TeamName   Status  QualifyingPosition  RacePosition  Points  Laps  AvgLapTime  NumPitStops  AvgTrackTemp  AvgAirTemp  RainDuringRace  SafetyCarDeployed   Time_s     Time_str  GapToWinner_s  WinnerRaceTime_s  FinishedFlag
 2021      1 Bahrain Grand Prix Bahrain          HAM   Lewis Hamilton        Mercedes Finished                 2.0           1.0    25.0  56.0   97.586200          2.0      27.62782   20.538346           False               True 5523.897 01:32:03.897          0.000          5523.897             1
 2021      1 Bahrain Grand Prix Bahrain          VER   Max Verstappen Red Bull Racing Finished                 1.0           2.0    18.0  56.0   97.575291          2.0      27.6278

In [86]:
train_data.dtypes
(
    train_data[train_data['RainDuringRace'] == False]  
    .groupby('TrackName')['AvgLapTime']    
    .mean()
    .sort_values()
    .apply(_fmt_hms_ms)
)


TrackName
Austrian Grand Prix          00:01:12.574
Styrian Grand Prix           00:01:13.933
Dutch Grand Prix             00:01:17.189
Monaco Grand Prix            00:01:17.447
São Paulo Grand Prix         00:01:19.687
Canadian Grand Prix          00:01:22.890
Mexico City Grand Prix       00:01:25.141
Portuguese Grand Prix        00:01:27.638
Spanish Grand Prix           00:01:28.242
Emilia Romagna Grand Prix    00:01:30.593
Italian Grand Prix           00:01:31.276
Australian Grand Prix        00:01:31.668
Abu Dhabi Grand Prix         00:01:33.330
British Grand Prix           00:01:35.167
Turkish Grand Prix           00:01:35.797
Saudi Arabian Grand Prix     00:01:39.906
Bahrain Grand Prix           00:01:41.459
French Grand Prix            00:01:41.564
United States Grand Prix     00:01:43.987
Azerbaijan Grand Prix        00:01:51.342
Name: AvgLapTime, dtype: object

In [90]:
track_avg_speeds = {}
unique_tracks = train_data['TrackName'].unique()

for track in unique_tracks:
    mask = (train_data['TrackName'] == track) & (train_data['RainDuringRace'] == False)
    track_avg_speeds[track] = train_data[mask]['AvgLapTime'].mean()

print(track_avg_speeds)

{'Bahrain Grand Prix': 101.45865567960419, 'Emilia Romagna Grand Prix': 90.5926006491197, 'Portuguese Grand Prix': 87.63769235904721, 'Spanish Grand Prix': 88.24215004784568, 'Monaco Grand Prix': 77.44665136232187, 'Azerbaijan Grand Prix': 111.34192204731403, 'French Grand Prix': 101.56437462653813, 'Styrian Grand Prix': 73.93315854644348, 'Austrian Grand Prix': 72.57358236837692, 'British Grand Prix': 95.16720259413009, 'Hungarian Grand Prix': nan, 'Belgian Grand Prix': nan, 'Dutch Grand Prix': 77.18882776097315, 'Italian Grand Prix': 91.27604197207627, 'Russian Grand Prix': nan, 'Turkish Grand Prix': 95.79668036038372, 'United States Grand Prix': 103.9871068754724, 'Mexico City Grand Prix': 85.14129938006842, 'São Paulo Grand Prix': 79.6873411247539, 'Qatar Grand Prix': nan, 'Saudi Arabian Grand Prix': 99.9057670883021, 'Abu Dhabi Grand Prix': 93.32971996743632, 'Australian Grand Prix': 91.66839407266208, 'Miami Grand Prix': nan, 'Canadian Grand Prix': 82.89026159743331}


In [96]:
tracks_2023 = [
    'Bahrain Grand Prix',
    'Saudi Arabian Grand Prix', 
    'Australian Grand Prix',
    'Azerbaijan Grand Prix',
    'Miami Grand Prix',
    'Monaco Grand Prix',
    'Spanish Grand Prix',
    'Canadian Grand Prix',
    'Austrian Grand Prix',
    'British Grand Prix',
    'Hungarian Grand Prix',
    'Belgian Grand Prix',
    'Dutch Grand Prix',
    'Italian Grand Prix',
    'Singapore Grand Prix',  
    'Japanese Grand Prix',  
    'Qatar Grand Prix',
    'United States Grand Prix',
    'Mexico City Grand Prix',
    'São Paulo Grand Prix',
    'Las Vegas Grand Prix', 
    'Abu Dhabi Grand Prix'
]

train_data_2023 = train_data[train_data['TrackName'].isin(tracks_2023)]

In [114]:
unique_tracks = train_data_2023['TrackName'].unique()

sprint_races = [(2021, 'British Grand Prix'), (2021, 'Italian Grand Prix'), (2021, 'São Paulo Grand Prix'), (2022, 'Austrian Grand Prix'), (2022, 'São Paulo Grand Prix')]

is_sprint = False
for year, track in sprint_races:
    is_sprint |= ((train_data['Year'] == year) & (train_data['TrackName'] == track))


train_data_no_sprints = train_data[~is_sprint]

overtake_difficulty_clean = {}
street_circuits = ['Monaco Grand Prix', 'Azerbaijan Grand Prix', 'Saudi Arabian Grand Prix', 'Miami Grand Prix']


for track in unique_tracks:
    mask = ((train_data_no_sprints['TrackName'] == track) & 
            (train_data_no_sprints['FinishedFlag'] == 1))
    
    data = train_data_no_sprints[mask][['QualifyingPosition', 'RacePosition']]
    data = data.dropna()
    
    
    if len(data) > 5:
        correlation = data['QualifyingPosition'].corr(data['RacePosition'])
        overtake_difficulty_clean[track] = correlation
        
        
    else:
        if track in street_circuits:
            overtake_difficulty_clean[track] = 0.8
        else:
            overtake_difficulty_clean[track] = 0.6
        
print(overtake_difficulty_clean)

{'Bahrain Grand Prix': 0.876258067120981, 'Spanish Grand Prix': 0.8698540564281028, 'Monaco Grand Prix': 0.8814831792531322, 'Azerbaijan Grand Prix': 0.6269010633128796, 'Austrian Grand Prix': 0.6417609701883894, 'British Grand Prix': 0.7219697431870115, 'Hungarian Grand Prix': 0.28981719065185746, 'Belgian Grand Prix': 0.5329293868183337, 'Dutch Grand Prix': 0.6, 'Italian Grand Prix': 0.6, 'United States Grand Prix': 0.8411910241920599, 'Mexico City Grand Prix': 0.6, 'São Paulo Grand Prix': 0.6, 'Qatar Grand Prix': 0.6601320396132047, 'Saudi Arabian Grand Prix': 0.8392343895515995, 'Abu Dhabi Grand Prix': 0.6818673434061754, 'Australian Grand Prix': 0.9251011822888074, 'Miami Grand Prix': 0.6305016117798247, 'Canadian Grand Prix': 0.49626193359530535}


In [115]:
# Create track features with just 2 features
track_features = pd.DataFrame({'TrackName': list(overtake_difficulty_clean.keys()), 'overtaking_difficulty': list(overtake_difficulty_clean.values())})

street_circuits = ['Monaco Grand Prix', 'Azerbaijan Grand Prix', 'Saudi Arabian Grand Prix', 'Miami Grand Prix']
track_features['is_street'] = track_features['TrackName'].isin(street_circuits).astype(int)

print(track_features.head())

               TrackName  overtaking_difficulty  is_street
0     Bahrain Grand Prix               0.876258          0
1     Spanish Grand Prix               0.869854          0
2      Monaco Grand Prix               0.881483          1
3  Azerbaijan Grand Prix               0.626901          1
4    Austrian Grand Prix               0.641761          0


In [116]:
train_data_with_features = train_data_no_sprints.merge(
    track_features, 
    on='TrackName', 
    how='left'
)

print(train_data_with_features[['TrackName', 'is_street', 'overtaking_difficulty']].head())

            TrackName  is_street  overtaking_difficulty
0  Bahrain Grand Prix        0.0               0.876258
1  Bahrain Grand Prix        0.0               0.876258
2  Bahrain Grand Prix        0.0               0.876258
3  Bahrain Grand Prix        0.0               0.876258
4  Bahrain Grand Prix        0.0               0.876258


In [117]:
print(train_data_with_features[train_data_with_features['is_street'] == 1]['TrackName'].unique())

['Monaco Grand Prix' 'Azerbaijan Grand Prix' 'Saudi Arabian Grand Prix'
 'Miami Grand Prix']
